# Predicting house prices using k-nearest neighbors regression

In this notebook, you will implement k-nearest neighbors regression. You will:

* Find the k-nearest neighbors of a given query input
* Predict the output for the query input using the k-nearest neighbors
* Choose the best value of k using a validation set

## Import Moduels

In [1]:
import numpy as np
import pandas as pd

## Load Data

In [2]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

sales = pd.read_csv('kc_house_data_small.csv', dtype = dtype_dict)
train = pd.read_csv('kc_house_data_small_train.csv', dtype = dtype_dict)
test = pd.read_csv('kc_house_data_small_test.csv', dtype = dtype_dict)
validation = pd.read_csv('kc_house_data_validation.csv', dtype = dtype_dict)

To efficiently compute pairwise distances among data points, we will convert the SFrame (or dataframe) into a 2D Numpy array. First import the numpy library and then copy and paste `get_numpy_data()` (or equivalent). The function takes a dataset, a list of features (e.g. [‘sqft_living’, ‘bedrooms’]) to be used as inputs, and a name of the output (e.g. `price`). It returns a `features_matrix` (2D array) consisting of a column of ones followed by columns containing the values of the input features in the data set in the same order as the input list. It also returns an `output_array`, which is an array of the values of the output in the dataset (e.g. `price`).

In [3]:
def get_numpy_data(data_frame, features, output):
    data_frame = data_frame.assign(constant = 1)
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    feature_df = data_frame.loc[:, features]
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = feature_df.values
    output_df = data_frame.loc[:, output]
    output_array = output_df.values
    
    return(feature_matrix, output_array)

Similarly, copy and paste the `normalize_features` function (or equivalent) from Module 5 (Ridge Regression). Given a feature matrix, each column is divided (element-wise) by its 2-norm. The function returns two items: (i) a feature matrix with normalized columns and (ii) the norms of the original columns.

In [4]:
def normalize_features(features):
    norms = np.linalg.norm(features, axis = 0)
    normalized_features = features / norms
    return (normalized_features, norms)

Using `get_numpy_data` (or equivalent), extract numpy arrays of the training, test, and validation sets. Use below `feature_list`.

In [5]:
feature_list = ['bedrooms',  
                'bathrooms',  
                'sqft_living',  
                'sqft_lot',  
                'floors',
                'waterfront',  
                'view',  
                'condition',  
                'grade',  
                'sqft_above',  
                'sqft_basement',
                'yr_built',  
                'yr_renovated',  
                'lat',  
                'long',  
                'sqft_living15',  
                'sqft_lot15']

In [6]:
features_train, output_train = get_numpy_data(train, feature_list, ['price'])
features_test, output_test = get_numpy_data(test, feature_list, ['price'])
features_valid, output_valid = get_numpy_data(validation, feature_list, ['price'])

In computing distances, it is crucial to normalize features. Otherwise, for example, the `sqft_living` feature (typically on the order of thousands) would exert a much larger influence on distance than the `bedrooms` feature (typically on the order of ones). We divide each column of the training feature matrix by its 2-norm, so that the transformed column has unit norm.

**IMPORTANT**: Make sure to store the norms of the features in the training set. The features in the test and validation sets must be divided by these same norms, so that the training, test, and validation sets are normalized consistently.

In [7]:
features_train, norms = normalize_features(features_train)

In [8]:
features_test = features_test / norms
features_valid = features_valid / norms

## Compute a single distance

To start, let's just explore computing the “distance” between two given houses. We will take our query house to be the first house of the test set and look at the distance between this house and the 10th house of the training set.

To see the features associated with the query house, print the first row (index 0) of the test feature matrix. You should get an 18-dimensional vector whose components are between 0 and 1. Similarly, print the 10th row (index 9) of the training feature matrix.

In [9]:
print(features_test[0])
print(features_train[9])

[ 0.01345102  0.01551285  0.01807473  0.01759212  0.00160518  0.017059
  0.          0.05102365  0.0116321   0.01564352  0.01362084  0.02481682
  0.01350306  0.          0.01345387 -0.01346922  0.01375926  0.0016225 ]
[ 0.01345102  0.01163464  0.00602491  0.0083488   0.00050756  0.01279425
  0.          0.          0.01938684  0.01390535  0.0096309   0.
  0.01302544  0.          0.01346821 -0.01346251  0.01195898  0.00156612]


**Quiz Question 1**: What is the Euclidean distance between the query house and the 10th house of the training set?

Note: Do not use the ‘np.linalg.norm’ function; use ‘np.sqrt’, ‘np.sum’, and the power operator (\*\*) instead. The latter approach is more easily adapted to computing multiple distances at once.

In [10]:
distance_qz1 = np.sqrt(np.sum((features_test[0] - features_train[9]) ** 2))
print(round(distance_qz1, 3))

0.06


Of course, to do nearest neighbor regression, we need to compute the distance between our query house and all houses in the training set.

To visualize this nearest-neighbor search, let's first compute the distance from our query house (`features_test[0]`) to the first 10 houses of the training set (`features_train[0:10]`) and then search for the nearest neighbor within this small set of houses. Through restricting ourselves to a small set of houses to begin with, we can visually scan the list of 10 distances to verify that our code for finding the nearest neighbor is working.

Write a loop to compute the Euclidean distance from the query house to each of the first 10 houses in the training set.

**Quiz Question 2**: Among the first 10 training houses, which house is the closest to the query house?

In [11]:
for i in range(10):
    distance_temp = np.sqrt(np.sum((features_test[0, :] - features_train[i, :]) ** 2))
    if i == 0:
        min_distance = distance_temp
        min_dist_index = i
    else:
        if distance_temp < min_distance:
            min_distance = distance_temp
            min_dist_index = i
        
print(min_distance)
print(min_dist_index)

0.052383627840220305
8


It is computationally inefficient to loop over computing distances to all houses in our training dataset. Fortunately, many of the numpy functions can be vectorized, applying the same operation over multiple values or vectors. We now walk through this process. (The material up to #13 is specific to numpy; if you are using other languages such as R or Matlab, consult relevant manuals on vectorization.)

Consider the following loop that computes the element-wise difference between the features of the query house (`features_test[0]`) and the first 3 training houses (`features_train[0:3]`):

In [12]:
for i in range(3):
    print(features_train[i]-features_test[0])
    # should print 3 vectors of length 18

[ 0.00000000e+00 -3.87821276e-03 -1.20498190e-02 -1.05552733e-02
  2.08673616e-04 -8.52950206e-03  0.00000000e+00 -5.10236549e-02
  0.00000000e+00 -3.47633726e-03 -5.50336860e-03 -2.48168183e-02
 -1.63756198e-04  0.00000000e+00 -1.70254220e-05  1.29876855e-05
 -5.14364795e-03  6.69281453e-04]
[ 0.00000000e+00 -3.87821276e-03 -4.51868214e-03 -2.26610387e-03
  7.19763456e-04  0.00000000e+00  0.00000000e+00 -5.10236549e-02
  0.00000000e+00 -3.47633726e-03  1.30705004e-03 -1.45830788e-02
 -1.91048898e-04  6.65082271e-02  4.23090220e-05  6.16364736e-06
 -2.89330197e-03  1.47606982e-03]
[ 0.00000000e+00 -7.75642553e-03 -1.20498190e-02 -1.30002801e-02
  1.60518166e-03 -8.52950206e-03  0.00000000e+00 -5.10236549e-02
  0.00000000e+00 -5.21450589e-03 -8.32384500e-03 -2.48168183e-02
 -3.13866046e-04  0.00000000e+00  4.70885840e-05  1.56292487e-05
  3.72914476e-03  1.64764925e-03]


The subtraction operator (-) in numpy is vectorized as follows:

In [13]:
print(features_train[0:3] - features_test[0])

[[ 0.00000000e+00 -3.87821276e-03 -1.20498190e-02 -1.05552733e-02
   2.08673616e-04 -8.52950206e-03  0.00000000e+00 -5.10236549e-02
   0.00000000e+00 -3.47633726e-03 -5.50336860e-03 -2.48168183e-02
  -1.63756198e-04  0.00000000e+00 -1.70254220e-05  1.29876855e-05
  -5.14364795e-03  6.69281453e-04]
 [ 0.00000000e+00 -3.87821276e-03 -4.51868214e-03 -2.26610387e-03
   7.19763456e-04  0.00000000e+00  0.00000000e+00 -5.10236549e-02
   0.00000000e+00 -3.47633726e-03  1.30705004e-03 -1.45830788e-02
  -1.91048898e-04  6.65082271e-02  4.23090220e-05  6.16364736e-06
  -2.89330197e-03  1.47606982e-03]
 [ 0.00000000e+00 -7.75642553e-03 -1.20498190e-02 -1.30002801e-02
   1.60518166e-03 -8.52950206e-03  0.00000000e+00 -5.10236549e-02
   0.00000000e+00 -5.21450589e-03 -8.32384500e-03 -2.48168183e-02
  -3.13866046e-04  0.00000000e+00  4.70885840e-05  1.56292487e-05
   3.72914476e-03  1.64764925e-03]]


Note that the output of this vectorized operation is identical to that of the loop above, which can be verified below:

In [14]:
# verify that vectorization works
results = features_train[0:3] - features_test[0]
print(results[0] - (features_train[0]-features_test[0]))
# should print all 0's if results[0] == (features_train[0]-features_test[0])
print(results[1] - (features_train[1]-features_test[0]))
# should print all 0's if results[1] == (features_train[1]-features_test[0])
print(results[2] - (features_train[2]-features_test[0]))
# should print all 0's if results[2] == (features_train[2]-features_test[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


Aside: it is a good idea to write tests like this snippet whenever you are vectorizing a complicated operation.

## Perform 1-nearest neighbor regression

Now that we have the element-wise differences, it is not too hard to compute the Euclidean distances between our query house and all of the training houses. First, write a single-line expression to define a variable `diff` such that `diff[i]` gives the element-wise difference between the features of the query house and the i-th training house.

To test your code, print `diff[-1].sum()`, which should be `-0.0934339605842`.

The next step in computing the Euclidean distances is to take these feature-by-feature differences in `diff`, square each, and take the sum over feature indices. That is, compute the sum of squared feature differences for each training house (row in `diff`).

By default, `np.sum` sums up everything in the matrix and returns a single number. To instead sum only over a row or column, we need to specifiy the `axis` parameter described in the `np.sum` documentation. In particular, `axis=1` computes the sum across each row.

In [15]:
diff = features_train - features_test[0, :]
np.sum(diff ** 2, axis = 1)


array([0.00363304, 0.00730492, 0.00378218, ..., 0.0032681 , 0.00325555,
       0.00341846])

In [16]:
# verify 
np.sum(diff**2, axis=1)[15]

0.0033070590284564457

In [17]:
# verify
np.sum(diff[15]**2)

0.0033070590284564453

Verify that the two expressions yield the same results. That is, the output for the 16th house in the training set is equivalent to having examined only the 16th row of `diff` and computing the sum of squares on that row alone.

With this result in mind, write a single-line expression to compute the Euclidean distances from the query to all the instances. Assign the result to variable `distances`.

**Hint**: don't forget to take the square root of the sum of squares.

**Hint**: `distances[100]` should contain `0.0237082324496`.

Now you are ready to write a function that computes the distances from a query house to all training houses. The function should take two parameters: (i) the matrix of training features and (ii) the single feature vector associated with the query.

In [18]:
distances = np.sqrt(np.sum((features_train - features_test[0, :]) ** 2, axis = 1))

In [19]:
distances[100] # verified

0.023708232416678195

In [20]:
def compute_distances(features_instances, feature_query):
    """
    :type features_instances: a 2-d numpy array
    :type features_query: a 1-d numpy array
    :rtype: a 1-d numpy array
    """
    distances = np.sqrt(np.sum((features_instances - feature_query) ** 2, axis = 1))
    return distances

**Quiz Question 3**: Take the query house to be third house of the test set (`features_test[2]`). What is the index of the house in the training set that is closest to this query house?

In [21]:
distances_qz3 = compute_distances(features_train, features_test[2, :])

min_dist_qz3 = distances_qz3.min()
min_dist_index_qz3 = distances_qz3.argmin()

print('Min distance is %.8f, with an index of %d' % (min_dist_qz3, min_dist_index_qz3))

Min distance is 0.00286050, with an index of 382


**Quiz Question 4**: What is the predicted value of the query house based on 1-nearest neighbor regression?

In [22]:
print('Predicted value is %.2f' % (output_train[min_dist_index_qz3, 0]))

Predicted value is 249000.00


## Perform k-nearest neighbor regression

Using the functions above, implement a function that takes in

* the value of `k`;
* the feature matrix for the instances; and
* the feature of the query

and returns the indices of the k closest training houses. For instance, with 2-nearest neighbor, a return value of `[5, 10]` would indicate that the 6th and 11th training houses are closest to the query house.

**Hint**: look at the documentation for [np.argsort](https://docs.scipy.org/doc/numpy/reference/generated/numpy.argsort.html).

In [23]:
def k_nearest_neighbors(k, features_train, feature_query):
    """
    :type k: an integer
    :type features_train: a 2-d numpy array
    :type feature_query: a 1-d numpy array
    :rtype: a 1-d numpy array
    """
    distances = compute_distances(features_train, feature_query)
    indices_sorted = np.argsort(distances)
    neighbors = indices_sorted[0:k]
    return neighbors

**Quiz Question 5**: Take the query house to be third house of the test set (`features_test[2]`). What are the indices of the 4 training houses closest to the query house?

In [24]:
neighbors_qz5 = k_nearest_neighbors(4, features_train, features_test[2, :])

In [25]:
neighbors_qz5

array([ 382, 1149, 4087, 3142])

Now that we know how to find the k-nearest neighbors, write a function that predicts the value of a given query house. For simplicity, take the average of the prices of the k nearest neighbors in the training set. The function should have the following parameters:

* the value of k;
* the feature matrix for the instances;
* the output values (prices) of the instances; and
* the feature of the query, whose price we’re predicting.

The function should return a predicted value of the query house.

**Hint**: you can extract multiple items from a numpy array using a list of indices. For instance, `output_train[[6,10]]` returns the output values (prices) of the 7th and 11th instances.

In [26]:
def predict_output_of_query(k, features_train, output_train, feature_query):
    """
    :type k: an integer
    :type features_train: a 2-d numpy array
    :type output_train: a 1-d numpy array
    :type feature_query: a 1-d numpy array
    :rtype: an float
    """
    neighbors = k_nearest_neighbors(k, features_train, feature_query)
    prediction = np.average(output_train[neighbors])
    return prediction

**Quiz Question 6**: Again taking the query house to be third house of the test set (`features_test[2]`), predict the value of the query house using k-nearest neighbors with `k=4` and the simple averaging method described and implemented above.

In [27]:
prediction_qz6 = predict_output_of_query(4, features_train, output_train[:, 0], features_test[2])

In [28]:
print(round(prediction_qz6, 0))

413988.0


Finally, write a function to predict the value of each and every house in a query set. (The query set can be any subset of the dataset, be it the test set or validation set.) The idea is to have a loop where we take each house in the query set as the query house and make a prediction for that specific house. The new function should take the following parameters:

* the value of k;
* the feature matrix for the training set;
* the output values (prices) of the training houses; and
* the feature matrix for the query set.

The function should return a set of predicted values, one for each house in the query set.

**Hint**: to get the number of houses in the test set, use the `.shape` field of the feature matrix. See the [documentation](https://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.ndarray.shape.html).

In [29]:
def predict_output(k, features_train, output_train, features_query):
    """
    :type output_train: 1-d numpy array
    :rtype preidctions: 1-d numpy array
    """
    rounds = features_query.shape[0]
    predictions = np.array([])
    for i in range(rounds):
        prediction = predict_output_of_query(k, features_train, output_train, features_query[i, :])
        predictions = np.append(predictions, prediction)
    return predictions

**Quiz Question 7**: Make predictions for the first 10 houses in the test set, using `k=10`. What is the index of the house in this query set that has the lowest predicted value? What is the predicted value of this house?

In [30]:
predictions_qz7 = predict_output(10, features_train, output_train[:, 0], features_test[0:10, :])

In [31]:
print('Index of the house that has the lowest predicted value is %d.' % (predictions_qz7.argmin()))
print('Lowest predicted value is %.2f.' % (predictions_qz7.min()))

Index of the house that has the lowest predicted value is 6.
Lowest predicted value is 350032.00.


## Choosing the best value of k using a validation set

There remains a question of choosing the value of k to use in making predictions. Here, we use a validation set to choose this value. Write a loop that does the following:

For k in `[1, 2, … 15]`:

* Make predictions for the VALIDATION data using the k-nearest neighbors from the TRAINING data.
* Compute the RSS on VALIDATION data

Report which k produced the lowest RSS on validation data.

In [32]:
from decimal import Decimal

k_qz8 = np.arange(1, 16, 1)
rss_qz8 = np.array([])

for k in k_qz8:
    predictions_tmp = predict_output(k, features_train, output_train[:, 0], features_valid)
    rss_tmp = np.sum((predictions_tmp - output_valid[:, 0]) ** 2)
    rss_qz8 = np.append(rss_qz8, rss_tmp)
    

In [33]:
print('The smallest RSS is %.2E, the K that produced it is %d.' %
     (rss_qz8.min(), k_qz8[rss_qz8.argmin()]))

The smallest RSS is 6.74E+13, the K that produced it is 8.


**Quiz Question 8**: What is the RSS on the TEST data using the value of k found above? To be clear, sum over all houses in the TEST set.

In [34]:
predictions_qz8 = predict_output(8, features_train, output_train[:, 0], features_test)
rss_qz8_test = np.sum((predictions_qz8 - output_test[:, 0]) ** 2)

print('The RSS on TEST data is %.2E' % (rss_qz8_test))

The RSS on TEST data is 1.33E+14
